In [1]:
import shap
from transformers import pipeline, AutoModelForSequenceClassification, AutoModelForMaskedLM, AutoTokenizer, DistilBertForSequenceClassification, AutoConfig
from datasets import load_from_disk
import torch
import numpy as np
import scipy as sp

# Masking

In [3]:
model = AutoModelForMaskedLM.from_pretrained('/datadrive_2/bnert-time-y').cuda()
#model = AutoModelForSequenceClassification.from_pretrained(pytorch_model.bin").cuda()
tokenizer = AutoTokenizer.from_pretrained("/datadrive_2/bnert-time-y")

In [4]:
preds = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0 , top_k=3)

In [9]:
mask = preds('[MASK] [DATE] her majesty the queen.')

[{'score': 0.04650492966175079,
  'token': 6899,
  'token_str': '1863',
  'sequence': '1863 her majesty the queen.'},
 {'score': 0.03965051844716072,
  'token': 8933,
  'token_str': '1853',
  'sequence': '1853 her majesty the queen.'},
 {'score': 0.03878328204154968,
  'token': 8421,
  'token_str': '1854',
  'sequence': '1854 her majesty the queen.'}]

In [6]:
explainer = shap.Explainer(preds)

In [ ]:
[{'text':'[MASK] [DATE] her majesty the queen.', 'label':'1847'}]

In [8]:
shap_values = explainer([{'text':'[MASK] [DATE] her majesty the queen.', 'label':'1847'}])

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
shap.plots.text(shap_values)

# Political Leaning

In [ ]:
!ls /datadrive_2/results_bnert-time-y/checkpoint-3000

In [ ]:

config = AutoConfig.from_pretrained("/datadrive_2/bNERT/results_bnert-time-y/checkpoint-3000")
model = DistilBertForSequenceClassification(config=config).cuda()
#model = AutoModelForSequenceClassification.from_pretrained(pytorch_model.bin").cuda()
tokenizer = AutoTokenizer.from_pretrained("/datadrive_2/bNERT/results_bnert-time-y/checkpoint-3000")

In [ ]:

pred = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0, return_all_scores=True)

In [ ]:
explainer = shap.Explainer(pred)

In [ ]:
cache_dir = '/datadrive_2/hf_cache/'
dataset = load_from_disk("/datadrive_2/HMD_context")
dataset = dataset['train']

In [ ]:
dataset

In [ ]:
dataset['text'][1000:1010]

In [ ]:
shap_values = explainer(dataset['text'][1000:1010])

In [ ]:
shap.plots.text(shap_values)

# Animacy

In [ ]:


# load a BERT sentiment analysis model
model = AutoModelForSequenceClassification.from_pretrained("/datadrive_2/bnert-bnert-time-y-animacy").cuda()
tokenizer = AutoTokenizer.from_pretrained(
    "/datadrive_2/bnert-bnert-time-y-animacy"
    )

In [ ]:


# define a prediction function
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=256, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

# build an explainer using a token masker
explainer = shap.Explainer(f, tokenizer)

In [ ]:

animacy_data = load_from_disk('/datadrive_2/animacy_split')
animacy_data_viz = animacy_data.remove_columns(['Unnamed: 0', 'Date', 'Sentence', 'SentenceCtxt', 'SentenceId', 'TargetExpression', 'animacy', 'humanness', 'st_year_sep', 'year_sep'])
animacy_data_viz = animacy_data_viz.rename_column('year_date','text')
animacy_data_viz 

In [ ]:
animacy_data_viz['test']['label']

In [ ]:
shap_values = explainer(animacy_data_viz['test'][:20])

In [ ]:
shap_values[3]

In [ ]:
# plot the first sentence's explanation
shap.plots.text(shap_values[17])

In [ ]:
shap.plots.bar(shap_values.max(0),50)

In [ ]:
shap_values.data[:]